In [96]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import torch
from transformers import pipeline
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import openpyxl
import re

In [63]:
# importing the final df of all parties.
df_final_tweets = pd.read_csv('../data/processed/df_final_v2.csv')
df_final_mentions = pd.read_csv('../data/processed/df_final_mentions_2.csv')

In [64]:
# creating just one dataframe
frames = [df_final_tweets, df_final_mentions]
df_total = pd.concat(frames, ignore_index=True)

In [75]:
df_total.drop(labels='Unnamed: 0', axis=1)

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,entities.hashtags,entities.urls,partido,tipo de post
0,Partido Popular,2021-04-17 10:29:44+00:00,1383367063666782213,👉 Javier Fernández-Lasquetty: “Todos los socia...,"[0, 252]",['Twitter Media Studio - LiveCut'],2,2,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [241, 252]}]",[],Partido Popular,publicación
1,Partido Popular,2021-04-17 10:25:21+00:00,1383365962888409090,👉 Javier Fernández-Lasquetty: “Libertad para t...,"[0, 163]",['Twitter Media Studio - LiveCut'],7,11,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [152, 163]}]",[],Partido Popular,publicación
2,Partido Popular,2021-04-17 10:16:32+00:00,1383363740356145161,📡 #EnDirecto👉 @IdiazAyuso atiende a los medios...,"[0, 103]",['Twitter Media Studio'],16,34,820639,5291,102263,"[{'text': 'EnDirecto', 'indices': [2, 12]}]","[{'url': 'https://t.co/N4H746boKu', 'expanded_...",Partido Popular,publicación
3,Partido Popular,2021-04-17 09:59:36+00:00,1383359481254268936,"🔵 En unos minutos, no te pierdas la intervenci...","[0, 179]",['Twitter Web App'],37,49,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [168, 179]}]","[{'url': 'https://t.co/6WGpUW8ul9', 'expanded_...",Partido Popular,publicación
4,Partido Popular,2021-04-17 06:14:00+00:00,1383302705779671044,☕️ Muy buenos días y #FelizSábado a todos 😊\n\...,"[0, 102]",['Twitter Web App'],8,26,820639,5291,102263,"[{'text': 'FelizSábado', 'indices': [21, 33]}]","[{'url': 'https://t.co/3quq6iOQY2', 'expanded_...",Partido Popular,publicación
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Vox_Hellin,2021-04-17 10:58:55+00:00,1383374409545576455,‼️Nosotros luchamos por todos los españoles si...,"[0, 158]",['Twitter for Android'],0,0,290,338,3340,"[{'text': 'ProtegeMadrid', 'indices': [125, 13...",[],Vox,mención
996,Francisco J.rey /🌹❤,2021-04-17 10:58:49+00:00,1383374384346267651,@Sirona33014070 @TaniaCrespo3 @LorenaCastro20 ...,"[60, 198]",['Twitter for Android'],0,0,1005,991,5732,[],[],Vox,mención
997,christian9516,2021-04-17 10:58:40+00:00,1383374347344125957,@TabarniaB Sería noticia que Vox no llenará el...,"[11, 63]",['Twitter for Android'],0,0,23,75,2859,[],[],Vox,mención
998,justi,2021-04-17 10:58:21+00:00,1383374264561061891,El doble pinchazo de Vox en Orcasitas: ni públ...,"[0, 87]",['Twitter for iPhone'],0,0,954,976,81113,[],"[{'url': 'https://t.co/aNGIIqo431', 'expanded_...",Vox,mención


## Machine learning model: BERT

This model classify text into 1, 2, 3, 4 or 5 starts. Being 1 the most negative and 5 the most positive. 

In [7]:
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

In [10]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [11]:
list_tweets = df_total['full_text'].tolist()

In [12]:
results = classifier(list_tweets)

In [13]:
for result in results[:10]:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: 1 star, with score: 0.6237
label: 5 stars, with score: 0.3315
label: 5 stars, with score: 0.3107
label: 5 stars, with score: 0.4093
label: 5 stars, with score: 0.6952
label: 1 star, with score: 0.7363
label: 1 star, with score: 0.658
label: 1 star, with score: 0.5843
label: 1 star, with score: 0.4707
label: 1 star, with score: 0.5427


In [132]:
sentiment_list = []
scores = []
for result in results:
    scores_list = result['score']
    scores.append(scores_list)
    
    for i in range(len(result)):
        results_list = result['label'][i]
        if results_list != " ":
            sentiment_list.append(results_list)

In [134]:
len(sentiment_list)

1000

In [123]:
for i in range(len(results)):
    results_list = result['label'][i]
    sentiment_list.append(results_list)

IndexError: string index out of range

In [135]:
df_total['sentimiento'] = sentiment_list
df_total['scores'] = scores

In [136]:
df_total.drop(labels='Unnamed: 0', axis=1)

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,entities.hashtags,entities.urls,partido,tipo de post,sentimiento,scores
0,Partido Popular,2021-04-17 10:29:44+00:00,1383367063666782213,👉 Javier Fernández-Lasquetty: “Todos los socia...,"[0, 252]",['Twitter Media Studio - LiveCut'],2,2,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [241, 252]}]",[],Partido Popular,publicación,1,0.623746
1,Partido Popular,2021-04-17 10:25:21+00:00,1383365962888409090,👉 Javier Fernández-Lasquetty: “Libertad para t...,"[0, 163]",['Twitter Media Studio - LiveCut'],7,11,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [152, 163]}]",[],Partido Popular,publicación,5,0.331548
2,Partido Popular,2021-04-17 10:16:32+00:00,1383363740356145161,📡 #EnDirecto👉 @IdiazAyuso atiende a los medios...,"[0, 103]",['Twitter Media Studio'],16,34,820639,5291,102263,"[{'text': 'EnDirecto', 'indices': [2, 12]}]","[{'url': 'https://t.co/N4H746boKu', 'expanded_...",Partido Popular,publicación,5,0.310655
3,Partido Popular,2021-04-17 09:59:36+00:00,1383359481254268936,"🔵 En unos minutos, no te pierdas la intervenci...","[0, 179]",['Twitter Web App'],37,49,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [168, 179]}]","[{'url': 'https://t.co/6WGpUW8ul9', 'expanded_...",Partido Popular,publicación,5,0.409290
4,Partido Popular,2021-04-17 06:14:00+00:00,1383302705779671044,☕️ Muy buenos días y #FelizSábado a todos 😊\n\...,"[0, 102]",['Twitter Web App'],8,26,820639,5291,102263,"[{'text': 'FelizSábado', 'indices': [21, 33]}]","[{'url': 'https://t.co/3quq6iOQY2', 'expanded_...",Partido Popular,publicación,5,0.695196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Vox_Hellin,2021-04-17 10:58:55+00:00,1383374409545576455,‼️Nosotros luchamos por todos los españoles si...,"[0, 158]",['Twitter for Android'],0,0,290,338,3340,"[{'text': 'ProtegeMadrid', 'indices': [125, 13...",[],Vox,mención,5,0.597425
996,Francisco J.rey /🌹❤,2021-04-17 10:58:49+00:00,1383374384346267651,@Sirona33014070 @TaniaCrespo3 @LorenaCastro20 ...,"[60, 198]",['Twitter for Android'],0,0,1005,991,5732,[],[],Vox,mención,1,0.557505
997,christian9516,2021-04-17 10:58:40+00:00,1383374347344125957,@TabarniaB Sería noticia que Vox no llenará el...,"[11, 63]",['Twitter for Android'],0,0,23,75,2859,[],[],Vox,mención,1,0.492922
998,justi,2021-04-17 10:58:21+00:00,1383374264561061891,El doble pinchazo de Vox en Orcasitas: ni públ...,"[0, 87]",['Twitter for iPhone'],0,0,954,976,81113,[],"[{'url': 'https://t.co/aNGIIqo431', 'expanded_...",Vox,mención,1,0.677282


In [44]:
# adding the starts to the dataframe for each tweet. 
# make a function to classify the tweets.

## Exporting the final data frame 

In [137]:
df_total.to_excel('../data/processed/df_total_SENTIMENT_2.xlsx')

In [138]:
df_total

,Unnamed: 0,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,entities.hashtags,entities.urls,partido,tipo de post,sentimiento,scores
0,0,Partido Popular,2021-04-17 10:29:44+00:00,1383367063666782213,👉 Javier Fernández-Lasquetty: “Todos los socia...,"[0, 252]",['Twitter Media Studio - LiveCut'],2,2,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [241, 252]}]",[],Partido Popular,publicación,1,0.623746
1,1,Partido Popular,2021-04-17 10:25:21+00:00,1383365962888409090,👉 Javier Fernández-Lasquetty: “Libertad para t...,"[0, 163]",['Twitter Media Studio - LiveCut'],7,11,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [152, 163]}]",[],Partido Popular,publicación,5,0.331548
2,2,Partido Popular,2021-04-17 10:16:32+00:00,1383363740356145161,📡 #EnDirecto👉 @IdiazAyuso atiende a los medios...,"[0, 103]",['Twitter Media Studio'],16,34,820639,5291,102263,"[{'text': 'EnDirecto', 'indices': [2, 12]}]","[{'url': 'https://t.co/N4H746boKu', 'expanded_...",Partido Popular,publicación,5,0.310655
3,3,Partido Popular,2021-04-17 09:59:36+00:00,1383359481254268936,"🔵 En unos minutos, no te pierdas la intervenci...","[0, 179]",['Twitter Web App'],37,49,820639,5291,102263,"[{'text': 'YoConAyuso', 'indices': [168, 179]}]","[{'url': 'https://t.co/6WGpUW8ul9', 'expanded_...",Partido Popular,publicación,5,0.409290
4,4,Partido Popular,2021-04-17 06:14:00+00:00,1383302705779671044,☕️ Muy buenos días y #FelizSábado a todos 😊\n\...,"[0, 102]",['Twitter Web App'],8,26,820639,5291,102263,"[{'text': 'FelizSábado', 'indices': [21, 33]}]","[{'url': 'https://t.co/3quq6iOQY2', 'expanded_...",Partido Popular,publicación,5,0.695196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,95,Vox_Hellin,2021-04-17 10:58:55+00:00,1383374409545576455,‼️Nosotros luchamos por todos los españoles si...,"[0, 158]",['Twitter for Android'],0,0,290,338,3340,"[{'text': 'ProtegeMadrid', 'indices': [125, 13...",[],Vox,mención,5,0.597425
996,96,Francisco J.rey /🌹❤,2021-04-17 10:58:49+00:00,1383374384346267651,@Sirona33014070 @TaniaCrespo3 @LorenaCastro20 ...,"[60, 198]",['Twitter for Android'],0,0,1005,991,5732,[],[],Vox,mención,1,0.557505
997,97,christian9516,2021-04-17 10:58:40+00:00,1383374347344125957,@TabarniaB Sería noticia que Vox no llenará el...,"[11, 63]",['Twitter for Android'],0,0,23,75,2859,[],[],Vox,mención,1,0.492922
998,98,justi,2021-04-17 10:58:21+00:00,1383374264561061891,El doble pinchazo de Vox en Orcasitas: ni públ...,"[0, 87]",['Twitter for iPhone'],0,0,954,976,81113,[],"[{'url': 'https://t.co/aNGIIqo431', 'expanded_...",Vox,mención,1,0.677282
